# Chapter 15 - Probabilistic Programming

In this chapter, we will learn about probabilistic programming, which is a programming paradigm that allows us to define probabilistic models and perform inference on them. We will use the PyMC3 library to define and perform inference on probabilistic models.

## 15.1 Relational Probability Models
- RPMs extend Bayesian networks by incorporating the expressiveness of first-order logic, allowing for a more natural representation of probabilistic models that involve relationships among a set of objects.
- Unlike traditional Bayesian networks, RPMs can represent and reason about an infinite set of possible worlds, making them suitable for complex domains where the number of objects or their relationships is not fixed.
### 15.1.1 Syntax and Semantics
- RPMs use a syntax similar to first-order logic, with constant, function, and predicate symbols to represent entities, their attributes, and the relationships between entities, respectively.
- A simple example provided involves an online book retailer wanting to evaluate books based on customer recommendations, highlighting how customer honesty and kindness, as well as book quality, can influence recommendations.
- The subsection describes how to specify the types of entities involved, their properties, and the dependencies between them using a formal syntax. This allows for the construction of a Bayesian network that represents the joint distribution of all variables in the model.
### 15.1.2 Example: Rating Player Skill Levels
- This subsection introduces an RPM for rating player skill levels in games, inspired by the Elo rating system. It outlines how a Bayesian model can estimate players' skill levels based on their performances in games.
- The model is extended to team games, where the overall performance of a team is considered the sum of individual performances. This demonstrates the flexibility of RPMs in modeling complex scenarios where the performance of individuals contributes to a collective outcome.
### 15.1.3 Inference in Relational Probability Models
- Discusses methods for performing inference in RPMs, emphasizing the challenges due to the potentially infinite number of possible worlds.
- It introduces the concept of "grounding" or "unrolling" the model to a finite Bayesian network by instantiating variables for a specific context, thus making inference computationally feasible.
- The section also touches on the use of context-specific independence and efficient inference algorithms to manage the complexity of RPMs.

These sections collectively showcase the power of RPMs in handling complex probabilistic models that involve relations among entities, demonstrating their application in diverse scenarios from book recommendations to player skill rating in games.

In [45]:
# let's do a simulation of Elo ratings for a chess tournament
# we will have 8 players, and they will play 7 rounds
# we will use the Elo rating system to calculate the ratings of the players
# we will constant K for this example
# we will use the following K values:
# K = 32 for a player with a rating of less than 2100
# K = 24 for a player with a rating of 2100 or more, but less than 2400
# K = 16 for a player with a rating of 2400 or more
# we will use the following formula to calculate the expected score of a player:
# E = 1 / (1 + 10^((R2 - R1) / 400))
# where R1 is the rating of player 1, and R2 is the rating of player 2
# we will use the following formula to calculate the new rating of a player:
# R = R + K * (S - E)   where S is the actual score of the player
# we will use the following scores:
# 1 for a win, 0.5 for a draw, and 0 for a loss
# we will use the following initial ratings:    
# 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400
# let's provide random results for the 7 rounds based on their expected scores
# let's calculate the new ratings of the players after the 7 rounds
# let's print the new ratings of the players
# let's plot the ratings of the players
# let's print the average rating of the players

import random
import numpy as np
import matplotlib.pyplot as plt

def expected_score(rating1, rating2):
    return 1 / (1 + 10**((rating2 - rating1) / 400))

def get_new_rating(rating, k, score, expected_score):
    return rating + k * (score - expected_score)

def k_value(rating):
    """Idea behind K rating is that the higher the rating, the more stable the player is, so the less the rating should change"""
    if rating < 2100:
        return 32
    elif rating < 2400:
        return 24
    else:
        return 16
    
# let's make a function that returns results of a single row as a string which we can print
# we do not want quotes around strings, so it should look like something like this  0 1 D X 1 0 0 0
def row_to_string(row):
    # if the score is 0.5 I want to print D
    # if the score is 0 I want to print 0
    # if the score is 1 I want to print 1
    # if the score is X I want to print X
    scores = []
    for score in row:
        if score == 0.5:
            scores.append('D')
        elif score == 0:
            scores.append('0')
        elif score == 1:
            scores.append('1')
        else:
            scores.append('X')
    return ' '.join(scores)
    
player_ratings = (2400, 2200, 2000, 1800, 1600, 1400, 1200, 1000)    
def simulate_elo_ratings(ratings = player_ratings, verbose=True):
    # ratings = [1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400]
    new_ratings = list(ratings)
    k_values = [k_value(rating) for rating in ratings]
    # results = np.zeros((8, 8))
    # instead of numpy I will use regular list
    results = [[0 for _ in range(8)] for _ in range(8)]
    for i in range(8):
        results[i][i] = 'X'
    for i in range(8):
        for j in range(8):
            if i != j:
                expected = expected_score(ratings[i], ratings[j])
                # so we compare this with a random number between 0 and 1
                result = random.random()
                # we can then make a decision based on the result if player exceeded or fell short of expectation
                precision = 0.05
                if abs(result - expected) < precision:
                    results[i][j] = 0.5
                    results[j][i] = 0.5
                elif result < expected:
                    results[i][j] = 1
                    results[j][i] = 0
                elif result > expected:
                    results[i][j] = 0
                    results[j][i] = 1
                # TODO think of other ways of generating drawish results
    for i in range(8):
        for j in range(8):
            if i != j: # we do not play ourselves!
                k = k_values[i]
                rating = ratings[i]
                score = results[i][j]
                expected = expected_score(rating, ratings[j])
                new_ratings[i] = get_new_rating(rating, k, score, expected)
    # let's print the tournament results and the new ratings
    # we want to also print original ratings of the players

    if verbose:
        # go through each player print old rating, scores, results and new rating in a single row
        for player, rating, new_rating, result in zip(range(8), ratings, new_ratings,  results):
            print(f"Player {player + 1}: {rating:8.2f} -> {new_rating:8.2f} -> {row_to_string(result)}")
    return ratings

ratings = simulate_elo_ratings()
# plt.plot(ratings)
# plt.show()
# print(f"Average rating: {sum(ratings) / 8}")


Player 1:  2400.00 ->  2400.01 -> X 1 1 1 1 1 1 1
Player 2:  2200.00 ->  2200.02 -> 0 X D 1 1 1 1 1
Player 3:  2000.00 ->  2000.10 -> 0 D X 1 1 1 1 1
Player 4:  1800.00 ->  1800.32 -> 0 0 0 X D 1 1 1
Player 5:  1600.00 ->  1600.98 -> 0 0 0 D X 1 1 1
Player 6:  1400.00 ->  1370.91 -> 0 0 0 0 0 X 1 0
Player 7:  1200.00 ->  1207.69 -> 0 0 0 0 0 0 X 1
Player 8:  1000.00 ->   992.31 -> 0 0 0 0 0 1 0 X


In [ ]:
# let's calculate the expected score when there is 200 point difference for example 2200 and 2000
print(expected_score(2200, 2000)) 


0.7597469266479578


### Improvements to Elo system

Ever since the original Arpad Elo invented the Elo system, there have been many improvements to the system. One of the most popular improvements is the Glicko system, which is an extension of the Elo system. The Glicko system introduces the concept of rating deviation, which represents the uncertainty in a player's skill level. The Glicko system also uses a more sophisticated update rule that takes into account the rating deviation and the volatility of a player's performance. These improvements make the Glicko system more accurate and robust than the original Elo system.

#### Links to papers on rating systems

- [Arpad Elo's original paper on the Elo system](https://gwern.net/doc/statistics/order/comparison/1978-elo-theratingofchessplayerspastandpresent.pdf)
- [Mark Glickman's site on ratings](http://www.glicko.net/glicko.html)

## 15.2 Open-Universe Probability Models
- Highlights the limitations of database semantics in representing real-world scenarios where the existence and identity of objects are uncertain, such as books with multiple ISBNs or users with multiple IDs (sybils).
- Introduces the concept of Open-Universe Probability Models (OUPMs) that handle existence and identity uncertainties, extending first-order logic to probabilistic modeling.
### 15.2.1 Syntax and Semantics
- Explains how OUPMs allow for the representation of models where the existence of objects is part of the uncertainty being modeled. This includes the introduction of number statements and origin functions to specify conditional distributions over the number of objects.
- Discusses the formal semantics of OUPMs, where each object in the model is defined by its generation history, and the world is defined by the values of all number variables and basic random variables.
### 15.2.2 Inference in Open-Universe Probability Models
- Due to the potentially vast or unbounded nature of the models, traditional unrolling and exact inference are impractical. Approximate inference algorithms, such as Markov Chain Monte Carlo (MCMC), are necessary.
- MCMC algorithms explore the space of possible worlds, where a move can alter the relationships, add or subtract objects, and change the interpretations of constant symbols.
### 15.2.3 Examples 
- **Citation Matching:**  Discusses the challenge of identifying whether different citations refer to the same paper, highlighting the complexity of modeling and inferring in scenarios with ambiguous object identities. 
- **Nuclear Treaty Monitoring:**  Describes the use of OUPMs in monitoring seismic events to ensure compliance with the Comprehensive Nuclear-Test-Ban Treaty, showcasing how OUPMs can model and infer the occurrence of natural and man-made seismic events.

This subchapter emphasizes the significance of modeling in scenarios where not all objects are known or identifiable, a common situation in many real-world applications. It demonstrates the utility of OUPMs in handling such complexity, from academic citation matching to global treaty enforcement.

### Citation Matching with OUPM 
- **Challenge:**  Academic papers often cite other works in their bibliographies, but these citations are just strings of text without unique identifiers. The goal is to identify which citations refer to the same academic paper despite variations in syntax, spelling, punctuation, and the presence of errors. 
- **Generative Model Concept:**  The model begins by considering researchers who write papers, which in turn are cited by others. The complexity arises in the citation strings, which must be parsed and matched to actual papers despite the lack of standard identifiers and the presence of errors. 
- **Model Components:**  
- **Types:**  Researcher, Paper, Citation. 
- **Variables:**  Include properties like the name of a researcher (`Name(Researcher)`), the title of a paper (`Title(Paper)`), and the text of a citation (`Text(Citation)`). 
- **Origin Function:**  Defines relationships such as which researcher authored which paper. 
- **Number Statements:**  Express uncertainty about the number of entities, such as researchers and papers. For instance, the model may use a distribution to express the expected number of papers a researcher has written. 
- **Inference Goal:**  Given citation strings as evidence, the model performs probabilistic inference to identify the most probable mapping between citations and the actual papers they refer to. This involves reasoning under uncertainty to handle ambiguities in citation strings. 
- **Benefits:**  Applying OUPM to citation matching allows for a more accurate parsing and matching of citations to papers compared to ad hoc methods based on textual similarity. The probabilistic model can significantly reduce error rates by leveraging a generative process that accounts for the way citations are produced and the inherent uncertainties in identifying cited papers.

This example showcases the power of OUPMs in handling real-world problems involving uncertainty about the existence and identity of objects. By constructing a probabilistic model that captures the generative process of academic citations and employing inference algorithms, systems can more accurately organize and present citation information, enhancing the utility of digital academic libraries.

## 15.3 "Keeping Track of a Complex World" from "Artificial Intelligence: A Modern Approach" 

discusses the challenges of monitoring multiple objects in dynamic environments. It delves into the problem of data association and the uncertainty of matching observations to the correct objects over time. 

- **15.3.1 Example: Multitarget Tracking** : Focuses on radar tracking of multiple aircraft, where the primary challenge is identifying which radar blips correspond to which aircraft across sequential time steps. 

- **Challenges:**  The example highlights the challenges of data association, where the number of objects is unknown, and the observations are noisy and uncertain. It emphasizes the need for probabilistic models to handle these uncertainties and infer the correct associations over time.

- **15.3.2 Example: Traffic Monitoring** : Although not explicitly detailed in the provided content, this section likely explores the application of probabilistic models to monitor traffic flow, tracking multiple vehicles, and analyzing traffic patterns under varying conditions and uncertainties.

- **Key Takeaways:**
    - The examples illustrate the complexity of tracking multiple objects in dynamic environments, where the number of objects is unknown, and the observations are noisy and uncertain.
    - Probabilistic models are essential for handling these uncertainties and inferring the correct associations over time, enabling systems to make informed decisions in complex, dynamic scenarios.

These sections highlight the complexity of probabilistic modeling in scenarios where multiple entities contribute to the data observed, emphasizing the need for sophisticated algorithms to infer the correct associations and states of these entities over time.

## 15.4 Programs as Probability Models

Subchapter 15.4 "Programs as Probability Models" introduces the concept of utilizing programs to define probabilistic models, highlighting how probabilistic programming languages (PPLs) leverage this approach for complex model representation and inference.
### 15.4.1 Example: Reading Text 
- **Generative Programs for OCR** : Illustrates probabilistic modeling with a generative program designed for optical character recognition (OCR), particularly for reading degraded text. This could apply to texts damaged by water, smudging, or aging. The program can also break certain CAPTCHAs. 
- **Process** : A generative program produces degraded images containing sequences of letters, first by generating a sequence of letters and then rendering these into a 2D image with additive noise. 
- **Output** : Sample images demonstrate the variability in the number of letters, their identities, and the level of noise introduced, showcasing the model's application in recognizing text under various conditions of degradation.
### 15.4.2 Syntax and Semantics 
- **Execution Trace and Generative Program** : The core concept here is that a generative program, through its execution involving random choices, defines a set of possible worlds (execution traces). Each random choice in the program corresponds to a random variable in the associated probability model. 
- **Probability of Execution Traces** : The probability distribution over these execution traces is the product of the probabilities for each random choice made during the program's execution. This setup allows for the representation of any probability distribution that can be sampled by a probabilistic Turing machine. 
- **Conversion from OUPM to Generative Programs** : The section discusses how open-universe probability models (OUPMs) can be converted into corresponding generative programs, which make random choices based on the model's structure and generate data structures representing the objects, functions, and relations within the possible worlds.

### Examples of OCR tasks

- **OCR for Degraded Text** : The generative program for OCR can be applied to reading degraded text, such as water-damaged documents, smudged text, or aged manuscripts. The program's ability to generate and recognize text under various conditions of degradation makes it a versatile tool for text recovery and analysis.
- **Breaking CAPTCHAs** : The generative program can also be used to break certain CAPTCHAs, which are designed to distinguish between humans and bots. By generating and recognizing text in distorted images, the program can automate the process of solving CAPTCHAs, demonstrating its utility in various applications beyond traditional OCR tasks.

#### Worst CAPTCHAs ever

- Add some examples of bad CAPTCHAS


### 15.4.3 Inference Results

This section discusses the application of a probabilistic model to interpret images of letters degraded with additive noise, highlighting the use of Markov Chain Monte Carlo (MCMC) inference to derive interpretations of the degraded texts. 
- **Application to Degraded Images** : The model is tested on images of letters that have been significantly degraded, making it challenging even for human interpretation. The inference results from three independent MCMC runs are presented, showcasing how the model attempts to recover the original letter sequences from these images. 
- **Inference Quality and Uncertainty** :
- In all three MCMC runs, the model suggests a letter sequence with high certainty for the correct interpretation, demonstrating the posterior distribution's concentration on plausible interpretations.
- Upon further degradation of text (making it difficult to read), the model still identifies the correct number of letters, but uncertainties and misidentifications occur, such as misidentifying the first letter. 
- **Interpretation of Results** :
- The outcome might reflect the model's accuracy and the inherent uncertainties in the degraded letters. Errors and uncertainties in letter identification might be unavoidable due to the model's limitations or excessive noise in the input.
- Another possibility is inadequate mixing of the MCMC inference, suggesting that extended computation could yield different, potentially more accurate results. This highlights a fundamental challenge in probabilistic inference: balancing computational cost against the accuracy and confidence of the inference. 
- **Improving Inference** :
- The discussion points towards potential improvements in the model or the inference process itself to achieve better results, such as enhancing the text model, reducing noise, or optimizing the MCMC algorithm's proposal distribution.
- Utilizing bottom-up clues from images could inform better initial hypotheses, potentially improving inference efficiency and accuracy.

This section encapsulates the challenges and considerations in applying probabilistic models to real-world problems, emphasizing the importance of iterative refinement of models and inference methods to handle complex, uncertain data effectively.

### 15.4.4 Improving the Generative Program to Incorporate a Markov Model

This section discusses enhancing the generative program for optical character recognition (OCR) by incorporating a Markov model to account for the sequential nature of letter appearance in English words. This modification aims to produce more realistic sequences of letters that reflect actual word structures, improving the OCR's ability to interpret degraded text.

**Key Improvements:**  
- **Sequential Letter Generation:**  Unlike the original model, which generated letters independently, the improved program generates letters sequentially. Each letter is drawn based on the previous letter, using transition probabilities that are estimated from a reference list of English words. 
- **Use of Markov Model:**  The generative program employs a Markov model that captures the likelihood of one letter following another within words. This approach is more sophisticated than generating letters independently, as it incorporates knowledge about common letter pairs (bigrams) in English. 
- **Enhanced Realism in Letter Sequences:**  The introduction of the Markov model results in generated sequences of letters that are more likely to resemble parts of English words. This realism significantly improves the program's utility for OCR tasks, especially in interpreting text degraded by factors like smudging, water damage, or aging.

**Impact on Inference:**  
- **Inference with High-Noise Images:**  The updated generative program was tested on highly noisy images, and the inference results showed a noticeable improvement in recognizing plausible letter sequences, even though some uncertainty remained. 
- **Comparison with Independent-Letter Model:**  The Markov model's inference results were compared with those from the original model that treated each letter as independent. The Markov model demonstrated a better understanding of plausible English letter sequences, reflecting prior knowledge of letter pair probabilities.

This improvement showcases the modular nature of probabilistic programming languages (PPLs), allowing for straightforward updates to the underlying model to enhance performance. By leveraging the sequential nature of text, the improved model offers a significant step forward in the accuracy and realism of generated letter sequences for OCR tasks.

### 15.4.5 Inference in Generative Programs

This section explores the challenges and strategies of performing inference in generative programs, which are central to probabilistic programming languages (PPLs). Given the usually prohibitive cost or impossibility of exact inference in these programs, the section outlines alternative approaches for practical applications.

**Key Points:**  
- **Rejection Sampling and Likelihood Weighting:**  Simple methods like rejection sampling and likelihood weighting are discussed. Rejection sampling involves running the program multiple times, retaining only those execution traces that align with observed evidence, and aggregating the outcomes to estimate probabilities. Likelihood weighting modifies this approach by tracking the "weight" of each trace, representing the cumulative probability of observed values, to manage less likely data more effectively. 
- **Challenges with Difficult Cases:**  For scenarios where data are unlikely under the model or the model is highly complex, Markov Chain Monte Carlo (MCMC) becomes the preferred method. MCMC allows for sampling and modifying execution traces within the program, addressing some of the limitations of simpler methods by enabling more nuanced exploration of the probability landscape. 
- **Modification of Execution Traces:**  The adaptation of MCMC to generative programs involves careful consideration of how changes to an execution trace might impact the validity of subsequent computations within the trace, especially for conditional structures like if-statements. 
- **Improvements and Special-Purpose Proposals:**  The section also touches on ongoing research aimed at enhancing the efficiency and applicability of inference in PPLs. This includes developing better proposal distributions for MCMC, which can significantly speed up the inference process by focusing on more likely or informative parts of the model's probability space. 
- **Computational Efficiency:**  Efforts to improve computational efficiency include leveraging "lifted inference" techniques, which avoid redundant calculations by exploiting the structure of probabilistic models, and exploring hardware and algorithmic optimizations for faster processing.

## Chapter Summary 

chapter provides a comprehensive exploration of sophisticated representations for probability models, leveraging the foundations of both logic and programming. Here's a summary of the key points covered in the chapter: 
- **Relational Probability Models (RPMs):**  RPMs are grounded in the database semantics of first-order languages, offering a framework for defining probability models in contexts where all objects and their identities are explicitly known. In RPMs, the possible worlds are defined by instantiations of predicate symbols with objects, ensuring a finite set of such worlds. This approach facilitates compact modeling of scenarios with numerous objects while adeptly managing relational uncertainty. 
- **Open-Universe Probability Models (OUPMs):**  Extending beyond RPMs, OUPMs utilize the full semantics of first-order logic to introduce and handle novel forms of uncertainty, including those related to the identity and existence of objects. This allows for more flexible and comprehensive modeling of complex, uncertain domains where the set of objects is not fixed or fully known in advance. 
- **Generative Programs and Probabilistic Programming Languages (PPLs):**  The chapter introduces generative programs as a method to represent probability models, including OUPMs, as executable code within a probabilistic programming language. These programs define distributions over their execution traces, capturing a wide range of probabilistic behaviors and scenarios. PPLs are highlighted for their universal expressive power, enabling detailed and nuanced modeling of diverse probabilistic systems.

In essence, the chapter delineates a progression from structured, logic-based models to more dynamic, programmatic approaches to probabilistic modeling. This evolution accommodates increasingly complex and uncertain information, illustrating the versatility and depth of modern probabilistic programming methodologies.

## Bibliographical and Historical Notes section of Chapter 15 provides a rich historical context and outlines the evolution of the connection between probability and logic, tracing back to Leibniz's work in 1704. This journey through the development of probabilistic logic highlights several key milestones and contributions: 
- **Early Attempts to Merge Probability and Logic:**  Hailperin (1984) and Howson (2003) document the long-standing efforts to integrate probability with logic, dating back to Leibniz's "Nouveaux Essais." Initial attempts often involved assigning probabilities directly to logical sentences. 
- **Gaifman's Propositional Probability Logic (1964):**  Introduced a rigorous approach where probability assertions acted as constraints on distributions over possible worlds, analogous to how logical sentences constrain possible worlds. This framework allowed for the proving of relationships between probabilities of logical sentences. 
- **Expansion to First-Order Probability Logic:**  Gaifman (1964) and later Scott and Krauss (1966) extended these ideas to first-order logic, enabling more expressive models that accommodate a wider range of probabilistic reasoning tasks. 
- **Probabilistic Logic Programs and Databases:**  The concepts evolved into probabilistic logic programs and probabilistic databases, where probabilities were assigned to logical structures like Horn clauses or database tuples, influencing both AI and database research. 
- **Challenges of Defining Complete and Consistent Models:**  The notes discuss the difficulties in creating fully consistent probability models when probabilities are attached to sentences, highlighting the work of Gaifman (1964) and the approaches of Nilsson (1986) and Paskin (2002) towards resolving these issues. 
- **Markov Logic Networks (MLNs):**  Introduced by Richardson and Domingos (2006), MLNs emerged as a popular approach for handling relational data by combining maximum entropy models with first-order logic. 
- **Evolution of Probabilistic Programming Languages (PPLs):**  The section charts the development of PPLs from their conceptual foundations to practical implementations like BLOG, CHURCH, Gen, Pyro, and Edward, showcasing their application in various domains. 
- **Improvements in Inference Techniques:**  The notes acknowledge ongoing efforts to enhance inference in probabilistic programs, covering advancements in algorithmic efficiency, compilation techniques, and the use of deep learning for improved sampling methods.